# Investigate Network

Version 3

In [1]:
import networkx as nx
import time
import math
import numpy as np
import scipy.stats
import scipy.linalg as la
import itertools
import matplotlib.pyplot as plt
from matplotlib import animation

from matplotlib.collections import LineCollection
from fa2 import ForceAtlas2
from data.curved_edges import *

from mirai import *

In [ ]:
def draw_graphS1(G, i, groups, leaders, labels, converged):

    fig, ax = plt.subplots(figsize=(20,15))
    
    #pos = nx.spring_layout(G)
    pos = nx.kamada_kawai_layout(G)
    
    # manually define border colors
    lineWidthsList = [10] * len(groups)
    lineWidthsList[-1] = 5
    borderColorsList = ['orange', 'blue', 'olive', 'teal', 'cyan', 'navy', 'grey', 'apricot', 'lavender', 'pink'][:len(groups)]
    borderColors = dict(zip(leaders.keys(), borderColorsList))
    lineWidths = dict(zip(leaders.keys(), lineWidthsList))
    
    for lead_node, contiguous_nodes in leaders.items():
        
        contiguous_nodes = [ j for j in G.nodes if G.nodes[j]['contiguous'] == True ]
        non_contiguous_nodes = [ node for node in G.nodes if node not in contiguous_nodes ]
        contiguous_colors = [ cm.Pastel2(G.nodes[j]['random_label']) for j in G.nodes if j in contiguous_nodes ]
        non_contiguous_colors = [ cm.Pastel2(G.nodes[j]['random_label']) for j in G.nodes if j in non_contiguous_nodes ]
        
        # color borders of nodes contiguous to community leader
        # edgecolors = borderColors[lead_node], linewidths=lineWidths[lead_node], 
        nx.draw_networkx(G, pos=pos, nodelist = contiguous_nodes, node_shape = 'o',
                     labels=labels, with_labels=True, node_color = contiguous_colors, 
                     alpha = 1, edgecolors = 'yellow', connectionstyle='arc3, rad=0.1',
                     width=0.005, ax=ax)
        
        nx.draw_networkx(G, pos=pos, nodelist = non_contiguous_nodes, node_shape = 'o',
                     labels=labels, with_labels=True, node_color = non_contiguous_colors,
                     alpha = 0.5, connectionstyle='arc3, rad=0.1',
                     width=0.005, ax=ax)
        
        # draw initial leaders
        
        nx.draw_networkx(G, pos=pos, nodelist = [lead_node], node_shape = 'o',
                     labels=labels, with_labels=True, node_color = [cm.Pastel2(G.nodes[lead_node]['random_label'])], 
                     alpha = 1, edgecolors = borderColors[lead_node], linewidths = lineWidths[lead_node],
                     width=0.005, ax=ax)
        
    ax.text(0.95, 0.95, "n: "+str(ni)+"   groups: "+str(len(groups)), ha="right", va="top", transform=plt.gca().transAxes)
    ax.text(0.95, 0.90, "step: "+str(i), ha="right", va="top", transform=plt.gca().transAxes)
    if converged:
        ax.text(0.95, 0.85, "converged!", ha="right", va="top", transform=plt.gca().transAxes)

    path = 'images/img' + format(i, '03d') + '.png'
    plt.savefig(path)
    plt.close()
    
    return None

In [2]:
def prune_graph(G):
    number_removed = 0 
    G_temp = G.copy()
    for node in G_temp.nodes:
        if G.degree(node) == 0:
            G.remove_node(node)
            number_removed = number_removed + 1
    print("pruned", number_removed, "nodes from the graph")
    return G

In [3]:
def draw_graphS2(G, i, groups, leaders, labels, pos, converged):
    
    # Produce the curves and axis
    fig, ax = plt.subplots(figsize=(20,20))
    curves = curved_edges(G, pos)
    lc = LineCollection(curves, color='grey', alpha=0.05)
    
    # nodes
    leader_keys = list(leaders.keys())
    # max 9
    borderColorsList = ['orange', 'olive', 'teal', 'cyan', 'navy', 'grey', 'apricot', 'lavender', 'pink'][:len(leaders)]
    contiguous_nodes = [ j for j in G.nodes if G.nodes[j]['contiguous'] == True ]
    non_contiguous_nodes = [ node for node in G.nodes if node not in contiguous_nodes ]
    
    nx.draw_networkx_labels(G, pos, labels=labels)
    
    nx.draw_networkx_nodes(G, pos, nodelist = leader_keys,
                           node_size=500, edgecolors = borderColorsList, linewidths = 10, node_color='red', alpha=0.8)
    
    nx.draw_networkx_nodes(G, pos, nodelist = contiguous_nodes,
                           node_size=100, edgecolors = 'yellow', linewidths = 10, node_color='cyan', alpha=0.8)
    
    nx.draw_networkx_nodes(G, pos, nodelist = non_contiguous_nodes,
                            node_size=100, node_color='cyan', alpha=0.8)
    
    # edges
    plt.gca().add_collection(lc)
    plt.tick_params(axis='both',which='both',bottom=False,left=False,labelbottom=False,labelleft=False)
    
    # text
    ax.text(0.95, 0.95, "n: "+str(ni)+"   groups: "+str(len(groups)), ha="right", va="top", transform=plt.gca().transAxes)
    ax.text(0.95, 0.90, "step: "+str(i), ha="right", va="top", transform=plt.gca().transAxes)
    if converged:
        ax.text(0.95, 0.85, "converged!", ha="right", va="top", transform=plt.gca().transAxes)

    path = 'images/img' + format(i, '03d') + '.png'
    plt.savefig(path)
    #plt.show()
    plt.close()

In [5]:
# n: number of nodes in each network; k: number of additional subnetworks
ni, k = 100, 2
n = ni * k

# values under more general conditions
#c, p1, p2 = 1, c * math.log(n) / n, 0.6 / n

# theorem two values automatically calculated
c, c2 = 0.00044, 1.2
p1 = c2 * (1800 * c * math.log(n) / ni)**(1/4)
p2 = 1/c2 * (ni * p1**2) / (8 * n)
nq, q = ni * 1, 0.31
print(k, "communities. p values", theorem_2(n, ni, p1, p2, c), "p:", p1, "p':", p2, "nq:", nq, "q:", q)

draw = True
interesting_graph = False
iteration = 0
m = 0
stopping_time, ultimate_stopping_time = 20, 20
start_time = time.time()

i = 0 
iteration = iteration + 1
converged = False
groups = np.arange(ni*k).reshape(k,ni)
leaders = []

print("iteration attempt ", iteration)
    
G = initialize_graph(ni, p1)
    
if k > 1:
    for add_graph in range(k-1):
        G = incorporate_graph(G, ni, p1, p2, k)
            
# add background noise
G = incorporate_graph(G, nq, q, q, k)

# prune nodes with no edges
G = prune_graph(G)

forceatlas2 = ForceAtlas2()
pos = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=50)
G_init = G.copy()

while not converged or i <= stopping_time or i <= ultimate_stopping_time:
    converged, leaders, labels = animate(G, i, groups, leaders, converged, p1)

    if draw == True:
        draw_graphS2(G, i, groups, leaders, labels, pos, converged)
            
    i = i + 1
    m = i
    print("convergence on step", i, "?", converged)
        
    # break after more than stopping time / log(n) steps
    
    if i >= stopping_time:
        break
    
    

end_time = time.time()
run_time = end_time - start_time
print("The program took", run_time, "seconds to compute a graph of", k, "clusters in", m, "steps. Convergence:", converged)
nx.write_graphml(G_init, 'images/S2_n100_k' + str(k) + '.graphml')

100%|██████████| 50/50 [00:00<00:00, 1084.26it/s]

2 communities. p values True p: 0.5431215481815791 p': 0.015363594588497674 nq: 100 q: 0.31
iteration attempt  1
pruned 0 nodes from the graph


BarnesHut Approximation  took  0.01  seconds
Repulsion forces  took  0.02  seconds
Gravitational forces  took  0.00  seconds
Attraction forces  took  0.01  seconds
AdjustSpeedAndApplyForces step  took  0.00  seconds
convergence on step 1 ? False
convergence on step 2 ? False
convergence on step 3 ? False
convergence on step 4 ? True
convergence on step 5 ? True
convergence on step 6 ? True
convergence on step 7 ? True
convergence on step 8 ? True
convergence on step 9 ? True
convergence on step 10 ? True
convergence on step 11 ? True
convergence on step 12 ? True
convergence on step 13 ? True
convergence on step 14 ? True
convergence on step 15 ? True
convergence on step 16 ? True
convergence on step 17 ? False
convergence on step 18 ? False
convergence on step 19 ? False
convergence on step 20 ? False
convergence on step 21 ? False
The program took 79.78784513473511 seconds to compute a graph of 2 clusters in 21 steps. Convergence: False


In [ ]:
adj_G = nx.adjacency_matrix(G).todense()
eigen_val, eigen_vec = la.eig(adj_G)

In [ ]:
def path_length(G, N, p):
    avg_degree = N * p
    exp_avg_path = math.log(N) / math.log(avg_degree)
    emp_avg_path = nx.average_shortest_path_length(G)
    print("average path length\nexpected:", exp_avg_path,"\nempirical:", emp_avg_path)

def clustering_coefficient(G, p):
    emp_clustering = nx.average_clustering(G)
    print("clustering coefficient of a random erdos_renyi graph\nexpected:", p, "\nepirical:", emp_clustering)
    #print("expected:", p)
    #print("empirical:", emp_clustering)

def spectral_density(eigen_val):
    rho = [None]*len(eigen_val)
    x = [None]*len(eigen_val)
    y = [None]*len(eigen_val)
    for index, val in enumerate(eigen_val):
        if abs(val) < 2 * math.sqrt(n * p * (1-p)):
            rho[index] = math.sqrt(4 * n * p * (1-p) - val**2) / (2 * math.pi * n * p * (1-p))
        else:
            rho[index] = 0
        x[index] = (val / math.sqrt(n * p * (1-p))).real
        y[index] = (rho[index] * math.sqrt(n * p * (1-p))).real
    xy = sorted(list(zip(x, y)), key = lambda tup: tup[0])
    x, y = list(zip(*xy))
    plt.plot(x, y, color='black', linewidth=0.8, dashes=[2,2,10,2])
    plt.title("spectral density")
    return rho

def empirical_rho(rho, eigen_val, N ,p):
    emp_rho = [None]*len(eigen_val)
    a = 10
    avg_degree = N * p
    for index, val in enumerate(eigen_val):
        emp_rho[index] = val**(-2 * val**2 + a) * math.exp((1 + math.log(avg_degree)) * val**2)
    for i in range(len(emp_rho)):
        print(rho[i], emp_rho[i])

expected_rho = spectral_density(eigen_val)
empirical_rho(expected_rho, eigen_val, n, p1)
path_length(G_view, n, p1)
clustering_coefficient(G, p1)

In [ ]:
# n: number of nodes in each network; k: number of additional subnetworks
ni, k = 100, 1

n = ni * k

c, c2 = 0.00044, 1.2
p1 = c2 * (1800 * c * math.log(n) / ni)**(1/4)
p0 = c2 * (1800 * c * math.log(n) / ni)

In [ ]:
p0, p1

In [ ]:
.524-.044

In [ ]:
np.arange(ni*).reshape(k,ni)

In [ ]:
for node in G_view.nodes:
    print(G_view.nodes[node])